In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('train.csv', sep=';')

In [3]:
texts = df['text']

In [7]:
from pymystem3 import Mystem
morph = Mystem()

In [8]:
import re
import nltk
nltk.download('punkt')

def text_to_sent(t):
    text = BeautifulSoup(t).text.lower()
    tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    raw_sentences = [x.split(';') for x in raw_sentences]
    raw_sentences = sum(raw_sentences, [])
    sentences = [morph.lemmatize(x) for x in raw_sentences]
    return [[y for y in x if re.match('[а-яёa-z0-9]', y)]
            for x in sentences]

[nltk_data] Downloading package punkt to
[nltk_data]     /home/petr.ermakov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
text_to_sent(texts[0])

[['в',
  'молодой',
  'и',
  'дружный',
  'коллектив',
  'динамично',
  'развиваться',
  'компания',
  'требоваться',
  'разработчик',
  'ведущий',
  'разработчик',
  'oracle',
  'siebel',
  'crm',
  'требование',
  'опыт',
  'работа',
  'с',
  'oracle',
  'siebel',
  'crm',
  'от',
  '2',
  'год'],
 ['обязательно',
  'знание',
  'siebel',
  'tools',
  'runtime',
  'events',
  'workflow',
  'dvm',
  'eai',
  'eim',
  'escript',
  'sql'],
 ['желательный',
  'знание',
  'siebel',
  'loyalty',
  'siebel',
  'financial',
  'services',
  'cti'],
 ['технический', 'английский', 'язык'],
 ['высокий', 'технический', 'образование'],
 ['обязанность',
  'проектирование',
  'система',
  'и',
  'подготовка',
  'технический',
  'документация'],
 ['разработка',
  'функциональность',
  'и',
  'интеграция',
  'в',
  'соответствие',
  'с',
  'требование'],
 ['оптимизация',
  'производительность',
  'система',
  'анализ',
  'ошибка',
  'и',
  'сбой'],
 ['участие', 'во', 'встреча', 'с', 'бизнес', 'заказчик

In [10]:
from tqdm import tqdm
sentences = []
for text in tqdm(texts):
    sentences += text_to_sent(text)

100%|██████████| 31063/31063 [03:59<00:00, 129.48it/s]


In [12]:
for text in tqdm(pd.read_csv('test.csv', sep=';')['text']):
    sentences += text_to_sent(text)

100%|██████████| 31064/31064 [04:05<00:00, 126.75it/s]


In [13]:
len(sentences)

688302

In [14]:
from gensim.models.word2vec import Word2Vec 

num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

In [15]:
model.init_sims(replace=True)

In [16]:
model.most_similar(['сбербанк'])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('втб', 0.658713698387146),
 ('мтс', 0.6366663575172424),
 ('ростелеком', 0.6092284917831421),
 ('альфа-банк', 0.5917276740074158),
 ('тинькофф', 0.5750815868377686),
 ('процессинг', 0.5739871263504028),
 ('промсвязьбанк', 0.5734691619873047),
 ('втб24', 0.5692766308784485),
 ('газпром', 0.5632192492485046),
 ('финтех', 0.5550127625465393)]

In [18]:
import numpy as np

index2word_set = set(model.wv.index2word)

def text_to_vec(words):
    text_vec = np.zeros((300,), dtype="float32")
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model.wv[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec


texts_vecs = np.zeros((len(df['text']), 300), dtype="float32")

for i, text in enumerate(df['text']):
    s = text_to_sent(text)
    s = sum(s, [])
    texts_vecs[i] = text_to_vec(s)

In [19]:
from sklearn.metrics.pairwise import cosine_distances

m = cosine_distances(texts_vecs)

m[0][1:].argmin()

m[0][10639+1]

print(df['text'][0])

print(df['text'][10639+1])

<p>В молодой и дружный коллектив динамично развивающейся компании требуется Разработчик/Ведущий разработчик Oracle Siebel CRM</p> <p><strong>Требования:</strong></p> <ul> <li>Опыт работы с <strong>Oracle Siebel CRM от 2 лет</strong>;</li> <li>Обязательно знание: Siebel Tools, Runtime Events, Workflow, DVM, EAI, EIM, eScript, SQL;</li> <li>Желательные знания: Siebel Loyalty, Siebel Financial Services, CTI;</li> <li>Технический английский язык;</li> <li>Высшее техническое образование.</li> </ul> <p><strong>Обязанности:</strong></p> <ul> <li>Проектирование системы и подготовка технической документации;</li> <li>Разработка функциональности и интеграции в соответствии с требованиями;</li> <li>Оптимизация производительности системы, анализ ошибок и сбоев;</li> <li>Участие во встречах с бизнес заказчиками.</li> </ul> <p><strong>Условия:</strong></p> <ul> <li>Возможность профессионального и карьерного роста в динамично развивающейся компании;</li> <li>Заработная плата обсуждается индивидуально

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    texts_vecs, df['target'],
    test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

/home/petr.ermakov/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/petr.ermakov/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8949370793093356

In [21]:
from sklearn.cluster import KMeans

In [22]:
word_vectors = model.wv.syn0

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [23]:
word_vectors.shape

(27262, 300)

In [24]:
num_clusters = 8000

In [ ]:
kmeans_clustering = KMeans(n_clusters=num_clusters, n_jobs=-1)
idx = kmeans_clustering.fit_predict(word_vectors)

In [ ]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [ ]:
for cluster in range(0,30):
    print("\nCluster %d" % cluster)
    
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)